# Using fairlearn GridSearch with census data

This notebook shows how to use `fairlearn` and the Fairness dashboard to generate models for the Census dataset. This dataset is a classification problem - given a range of data about 32,000 individuals, predict whether their annual income is above or below fifty thousand dollars per year.

For the purposes of this notebook, we shall treat this as a loan decision problem. We will pretend that the label indicates whether or not each individual repaid a loan in the past. We will use the data to train a model to predict whether previously unseen individuals will repay a loan or not. The assumption is that the model predictions are used to decide whether an individual should be offered a loan.

We will first train a fairness-unaware model and show that it leads to unfair decisions under a specific notion of fairness called *demographic parity*. We then mitigate unfairness by applying the `GridSearch` algorithm from `fairlearn` package.

## Load and preprocess the data set

For simplicity, we import the data set from the `shap` package, which contains the data in a cleaned format. We start by importing the various modules we're going to use:

In [ ]:
import sys
sys.path.insert(0, "../")

from fairlearn.metrics import DemographicParity
from fairlearn.reductions import GridSearch
from fairlearn.reductions.grid_search.simple_quality_metrics import SimpleClassificationQualityMetric
from fairlearn.moments import MisclassificationError, DP

from sklearn import svm, neighbors, tree
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd
import shap

import numpy as np

shap.initjs()

We can now load and inspect the data from the `shap` package:

In [ ]:
X_raw, Y = shap.datasets.adult()
X_raw

We are going to treat the sex of each individual as a protected attribute (where 0 indicates female and 1 indicates male), and in this particular case we are going separate this attribute out and drop it from the main data. We then perform some standard data preprocessing steps to convert the data into a format suitable for the ML algorithms

In [ ]:
A = X_raw["Sex"]
X = X_raw.drop(labels=['Sex'],axis = 1)
X = pd.get_dummies(X)

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

le = LabelEncoder()
Y = le.fit_transform(Y)

Finally, we split the data into training and test sets:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test, A_train, A_test = train_test_split(X_scaled, 
                                                    Y, 
                                                    A,
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=Y)

# Work around indexing bug
X_train = X_train.reset_index(drop=True)
A_train = A_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
A_test = A_test.reset_index(drop=True)

## Training a fairness-unaware model

To show the effect of `fairlearn` we will first train a standard ML model that does not incorporate fairness For speed of demonstration, we use a simple logistic regression learner from `sklearn`:

In [ ]:
unmitigated_model = LogisticRegression(solver='liblinear', fit_intercept=True)

unmitigated_model.fit(X_train, Y_train)

We can load this model into the Fairness dashboard, and examine how it is unfair (there is a warning about AzureML since we are not yet integrated with that product):

In [ ]:
from azureml.contrib.explain.model.visualize import FairnessDashboard

FairnessDashboard([unmitigated_model,unmitigated_model,unmitigated_model], X_test, Y_test.tolist(), pd.DataFrame(A_test).values.tolist(), True, list(X_test.columns), [0, 1], ["Sex"])

We can see that the error rate for the second group is approximately three times that of the first. Since we are deciding whether a given individual gets a loan or not, the opportunity tab is more important and shows that the second group would also be offered loans approximately three times as often.

Despite the fact that we removed the feature from the training data, our model still discriminates based on sex. This demonstrates that simply ignoring a protected attribute when fitting a model rarely eliminates unfairness. There will generally be enough other features correlated with the removed attribute to lead to disparate impact.

## Mitigation with GridSearch

The `GridSearch` class in `fairlearn` implements a simplified version of the exponentiated gradient reduction of [Agarwal et al. 2018](https://arxiv.org/abs/1803.02453). The user supplies a standard ML learner, which is treated as a blackbox. `GridSearch` works by generating a sequence of relabellings and reweightings, and trains a model for each.

For this example, we specify demographic parity (on the protected attribute of sex) as the fairness metric. Demographic parity requires that individuals are offered the opportunity (are approved for a loan in this example) independent of membership in the protected class (i.e., females and males should be offered loans at the same rate). We are using this metric for the sake of simplicity; in general, the appropriate fairness metric will not be obvious.

The third argument (quality metric) is not relevant in our example. In general, it is used to select one among the models generated by grid search; however, we will instead examine all the models since they allow us to trace the Pareto curve of trade-offs between fairness and accuracy.

In [ ]:
sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   disparity_metric=DemographicParity(),
                   quality_metric=SimpleClassificationQualityMetric())

Our algorithms provide `fit()` and `predict()` methods, so they behave in a similar manner to other ML packages in Python. We do however have to specify two extra arguments to `fit()` - the column of protected attribute labels, and also the number of models to generate in our sweep.

After `fit()` completes, we extract the full set of models from the `GridSearch` object.

In [ ]:
sweep.fit(X_train, Y_train,
          aux_data=A_train,
          number_of_lagrange_multipliers=71)

models = [ z.model for z in sweep.all_results]

We could load these models into the Fairness dashboard now. However, the plot would be somewhat confusing due to their number. In this case, we are going to remove the models which are dominated in the error-disparity space by others from the sweep (note that the disparity will only be calculated for the protected attribute; other potentially protected attributes will not be mitigated). In general, one might not want to do this, since there may be other considerations beyond the strict optimisation of error and disparity (of the given protected attribute).

In [ ]:
errors, disparities = [], []
for m in models:
    classifier = lambda X: m.predict(X)
    
    error = MisclassificationError()
    error.init(X_train, A_train, pd.Series(Y_train))
    disparity = DP()
    disparity.init(X_train, A_train, pd.Series(Y_train))
    
    errors.append(error.gamma(classifier)[0])
    disparities.append(disparity.gamma(classifier).max())
    
all_results = pd.DataFrame( {"model": models, "error": errors, "disparity": disparities})

non_dominated = []
for row in all_results.itertuples():
    errors_for_lower_or_eq_disparity = all_results["error"][all_results["disparity"]<=row.disparity]
    if row.error <= errors_for_lower_or_eq_disparity.min():
        non_dominated.append(row.model)

Finally, we can put the non-dominated into the Fairness dashboard. We also add in the original, unmitigated model - it will be the one highlighted when the dashboard first loads.

In [ ]:
dashboard_models = [unmitigated_model]
dashboard_models.extend(non_dominated)


FairnessDashboard(dashboard_models, X_test, Y_test.tolist(), pd.DataFrame(A_test).values.tolist(), True, list(X_test.columns), [0, 1], ["Sex"])

We see a Pareto front forming - the set of models which represent optimal tradeoffs between error and disparity. In the ideal case, we would have a model at the origin - perfectly accurate and without any unfairness under demographic parity (with respect to the protected attribute "sex"). The Pareto front represents the closest we can come to this ideal based on our data and choice of learner. Note the range of the axes - the disparity axis covers more values than the error, so we can reduce disparity substantially with a smaller increase in error.

When we choose a different feature to assess the models on the "Model Comparison" tab (for example race), we can see that the smooth tradeoff disappears, with some of the models which are on the Pareto front for mitigation on the basis of sex becoming obviously sub-optimal.

By selecting multiple models in the "Model Comparison" tab and then changing to the "Fairness in Accuracy" tab, we can compare the performance of different models for each of the two classes. This comparison can be performed for any of the available attributes.

Finally, we can examine the "Fairness in Opportunity" tab. We can see that selecting fairer models is indeed equalising the opportunity (in this toy model, the frequency at which loans are offered) between the classes. We can also see that this is happening by decreasing the acceptance rate for the second group of the protected attribtue.

The user can then decide which model on this frontier best suits their required tradeoff between fairness and accuracy.